In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas
import pandas as pd
from shapely.geometry import Point
import warnings
from shapely.wkt import loads
import trackintel as ti
from tqdm import tqdm

In [2]:
stop_point_df = pd.read_csv('staypoints.csv')
stop_point_df

,id,user_id,started_at,finished_at,geom
0,0,1,2022-09-06 22:09:29-04:00,2022-09-06 22:19:08-04:00,POINT (-81.6582657950000055 29.6534941799999991)
1,1,1,2022-09-06 22:19:08-04:00,2022-09-06 22:22:07-04:00,POINT (-81.6840639999999922 29.6517550000000014)
2,2,1,2022-09-06 22:22:07-04:00,2022-09-06 22:31:37-04:00,POINT (-81.6582655899999992 29.6534943599999998)
3,3,1,2022-09-06 22:31:37-04:00,2022-09-06 22:34:49-04:00,POINT (-81.6840650000000181 29.6517550000000014)
4,4,1,2022-09-06 22:34:49-04:00,2022-09-06 22:37:57-04:00,POINT (-81.6582659999999976 29.6534939999999985)
...,...,...,...,...,...
311567,311567,23296,2022-10-25 01:49:08-04:00,2022-10-25 01:52:46-04:00,POINT (-81.2076100000000025 29.4761099999999985)
311568,311568,23296,2022-10-25 03:06:21-04:00,2022-10-25 03:09:12-04:00,POINT (-81.2044099999999958 29.4846100000000000)
311569,311569,23296,2022-10-25 03:09:12-04:00,2022-10-25 03:17:47-04:00,POINT (-81.2028099999999995 29.4776099999999985)
311570,311570,23296,2022-10-26 22:56:59-04:00,2022-10-26 23:07:58-04:00,POINT (-81.2047730000000030 29.4855580000000010)


In [3]:
retail_pd = pd.read_csv('retail.csv')
retail_pd

,Unnamed: 0,ObjectId,Store_Name,Address,Address_Li,City,State,Zip5,Zip4,County,Longitude,Latitude,geometry
0,193,194,Sunshine Food Mart #202,1402 NE 25th Ave,NaN,Ocala,FL,34470,4861.0,MARION,-82.103302,29.201019,POINT (-82.10330199999999 29.20101900000003)
1,432,433,Macedonia Store,6128 County Road 23c,NaN,Macclenny,FL,32063,3714.0,BAKER,-82.138069,30.333633,POINT (-82.13806899999997 30.333633000000077)
2,906,907,Kurdi Fresh Seafood Market,452 N Marion Ave,NaN,Lake City,FL,32055,2877.0,COLUMBIA,-82.637283,30.194494,POINT (-82.63728299999997 30.194494000000077)
3,933,934,Union Street Farmers Market,111 E University Ave,NaN,Gainesville,FL,32601,5425.0,ALACHUA,-82.323547,29.651918,POINT (-82.32354699999996 29.651918000000023)
4,997,998,Bell's Produce,1190 S Main St,NaN,Bell,FL,32619,2378.0,GILCHRIST,-82.862099,29.755329,POINT (-82.86209899999994 29.755329000000074)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,244891,244892,Soriya Food Market,3561 Emerson St,NaN,Jacksonville,FL,32207,5628.0,DUVAL,-81.628120,30.291224,POINT (-81.62811999999997 30.291224000000057)
1995,244897,244898,Sunshine Food Mart 248,3495 W Anthony Rd,NaN,Ocala,FL,34475,2674.0,MARION,-82.143723,29.222822,POINT (-82.14372299999997 29.222822000000065)
1996,244899,244900,Star Food Discount,5120 N Pearl St,NaN,Jacksonville,FL,32208,5117.0,DUVAL,-81.656944,30.370920,POINT (-81.65694399999995 30.37092000000007)
1997,245092,245093,Quick N Save #2,1010 S Pine Ave,NaN,Ocala,FL,34471,995.0,MARION,-82.140869,29.178795,POINT (-82.14086899999995 29.178795000000036)


In [4]:
def is_close(dist, latA1, lonA2, latB1, lonB2):
    # Create shapely Point objects from the locations
    point1 = Point(lonA2, latA1)
    point2 = Point(lonB2, latB1)
    
    # Calculate distance between points in meters
    distance = point1.distance(point2) * 111319.9
    
    # Return True if distance is less than or equal to dist meters
    return distance <= dist

In [5]:
count = 0
warnings.simplefilter(action='ignore', category=Warning)
data_to_append_list = []
for sp in tqdm(stop_point_df.itertuples(), total=len(stop_point_df)):
    p = loads(sp[5])
    lat = p.y
    lon = p.x
    
    for row in retail_pd.itertuples():
        if is_close(200, float(row[12]), float(row[11]), lat, lon):
            count += 1
            
            data_to_append = {'id': sp[1], 'user_id': sp[2], 'started_at': sp[3], 'finished_at': sp[4], 'lat': lat, 'lon': lon, 'retail_id': row[1], 
                                'retail_lat': float(row[12]), 'retail_lon': float(row[11])}
            data_to_append_list.append(data_to_append)
            
            break

new_df = pd.DataFrame.from_records(data_to_append_list)
print('Point close to retail: ', count)

100%|██████████| 311572/311572 [4:03:51<00:00, 21.30it/s]  


Point close to retail:  40602


In [6]:
new_df

,id,user_id,started_at,finished_at,lat,lon,retail_id,retail_lat,retail_lon
0,7,1,2022-09-10 02:09:19-04:00,2022-09-10 02:11:50-04:00,29.655040,-81.657860,145695,29.655165,-81.656845
1,15,1,2022-09-26 02:07:17-04:00,2022-09-26 02:19:51-04:00,29.655035,-81.657880,145695,29.655165,-81.656845
2,17,2,2022-09-05 20:01:05-04:00,2022-09-05 20:04:13-04:00,29.651755,-82.339455,70228,29.653463,-82.339195
3,19,2,2022-09-07 20:02:10-04:00,2022-09-07 20:06:00-04:00,29.651742,-82.339464,70228,29.653463,-82.339195
4,21,2,2022-09-08 20:01:10-04:00,2022-09-08 20:04:18-04:00,29.651755,-82.339455,70228,29.653463,-82.339195
...,...,...,...,...,...,...,...,...,...
40597,311515,23285,2022-10-20 07:36:51-04:00,2022-10-20 07:46:01-04:00,29.599455,-82.087014,16846,29.598845,-82.086830
40598,311520,23285,2022-10-20 08:02:46-04:00,2022-10-20 08:06:32-04:00,29.265735,-82.192842,164487,29.267069,-82.192276
40599,311523,23285,2022-10-22 02:50:59-04:00,2022-10-22 03:00:35-04:00,29.407710,-82.248080,18329,29.406757,-82.248161
40600,311551,23285,2022-10-30 01:31:03-04:00,2022-10-30 01:33:29-04:00,30.002802,-82.597425,30766,30.001783,-82.597435


In [7]:
print(len(new_df['user_id'].unique()))

3563


In [8]:
new_df.to_csv('food_inference_200.csv')